In [1]:
from algos import libman, seidel, relaxation

import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import math

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

In [3]:
def select_solver(scheme):
    solver = None
    if scheme == 0:
        solver = libman
    elif scheme == 1:
        solver = seidel
    elif scheme == 2:
        solver = relaxation
    else:
        raise ValueError

    return solver

In [4]:
y_begin_widget = widgets.Text('0.0')
y_end_widget = widgets.Text(str(np.pi / 2))

current_y_widget = widgets.FloatSlider(min=0.0, max=1.0, step=0.01)

def update_current_y_range(*args):
    try:
        begin = float(y_begin_widget.value)
        end = float(y_end_widget.value)
        current_y_widget.min = begin
        current_y_widget.max = end
        current_y_widget.step = (end - begin) / 100
    except Exception:
        print('error')
        
y_begin_widget.observe(update_current_y_range, 'value')
y_end_widget.observe(update_current_y_range, 'value')

x_begin_widget = widgets.Text('0.0')
x_end_widget = widgets.Text(str(np.pi / 2))

current_x_widget = widgets.FloatSlider(min=0.0, max=1.0, step=0.01)

def update_current_x_range(*args):
    try:
        begin = float(x_begin_widget.value)
        end = float(x_end_widget.value)
        current_x_widget.min = begin
        current_x_widget.max = end
        current_x_widget.step = (end - begin) / 100
    except Exception:
        print('error')
        
x_begin_widget.observe(update_current_x_range, 'value')
x_end_widget.observe(update_current_x_range, 'value')

def solve(scheme, a, b, x_begin, x_end, y_begin, y_end, h1, h2, w, current_x, current_y):
    solver = select_solver(scheme) 
    try:
        a = float(a) 
        b = float(b)

        x_begin = float(x_begin)
        x_end = float(x_end)

        y_begin = float(y_begin)
        y_end = float(y_end)
        
        h1 = float(h1)
        h2 = float(h2)
        
        w = float(w)
    except Exception:
        print('Please, recheck your input')
        return

    def phi1(y):
        return np.cos(y)
    
    def phi2(y):
        return 0

    def psi1(x):
        return np.exp(-x) * np.cos(x)
    
    def psi2(x):
        return 0

    def u_ref(x, y):
        return np.exp(-x) * np.cos(x) * np.cos(y)
    
    nx = round((x_end - x_begin) / h1) + 1
    ny = round((y_end - y_begin) / h2) + 1

    x = np.linspace(x_begin, x_end, nx)
    y = np.linspace(y_begin, y_end, ny)
    
    if solver == relaxation:
        u, iters = solver(a, b, h1, h2, x_begin, x_end, y_begin, y_end, phi1, phi2, psi1, psi2, w)
    else:
        u, iters = solver(a, b, h1, h2, x_begin, x_end, y_begin, y_end, phi1, phi2, psi1, psi2)

    print(f'iterations taken: {iters}')
        
    n1 = math.floor((current_x - x_begin) / h1)
    n2 = math.floor((current_y - y_begin) / h2)
    
    u_ref_vals_x = u_ref(x, y[n2])
    u_ref_vals_y = u_ref(x[n1], y)
    
    fig, ax = plt.subplots()
    ax.plot(x, u[:, n2], 'g', label='result')
    ax.plot(x, u_ref_vals_x, 'r', label='reference')
    ax.legend()

    ax.set(xlabel='x', ylabel='u',
        title='u(x, y_fix)')
    ax.grid()

    plt.show()
    
    fig, ax = plt.subplots()
    ax.plot(x, np.absolute(u[:, n2] - u_ref_vals_x), 'r', label='error')
    ax.legend()

    ax.set(xlabel='x', ylabel='u',
        title='error(x)')
    ax.grid()

    plt.show()
    
    fig, ax = plt.subplots()
    ax.plot(y, u[n1], 'g', label='result')
    ax.plot(y, u_ref_vals_y, 'r', label='reference')
    ax.legend()

    ax.set(xlabel='y', ylabel='u',
        title='u(x_fix, y)')
    ax.grid()

    plt.show()

    fig, ax = plt.subplots()
    ax.plot(y, np.absolute(u[n1] - u_ref_vals_y), 'r', label='error')
    ax.legend()

    ax.set(xlabel='y', ylabel='u',
        title='error(y)')
    ax.grid()

    plt.show()
    
    hxs = []
    hys = []
    
    n_0 = 5
    n_n = 15
    
    for i in range(n_0, n_n):
        cur_hx = (x_end - x_begin) / (i - 1)
        hxs.append(cur_hx)
        cur_hy = (y_end - y_begin) / (i - 1)
        hys.append(cur_hy)
    
    errs = []
    for hx, hy in zip(hxs, hys):
        max_err = 0.0
        if solver == relaxation:
            u, _ = solver(a, b, hx, hy, x_begin, x_end, y_begin, y_end, phi1, phi2, psi1, psi2, w)
        else:
            u, _ = solver(a, b, hx, hy, x_begin, x_end, y_begin, y_end, phi1, phi2, psi1, psi2)
            
        nx = round((x_end - x_begin) / hx) + 1
        ny = round((y_end - y_begin) / hy) + 1

        y = np.linspace(y_begin, y_end, ny)

        for k in range(nx):
            u_ref_vals = u_ref(x_begin + k * hx, y)
            err = np.max(np.abs(u[k] - u_ref_vals))
            if err > max_err:
                max_err = err
                
        errs.append(max_err)

    errs = np.array(errs)
    
    fig, ax = plt.subplots()

    ax.plot(np.log(hxs), np.log(errs), 'r', label='max_error')
    ax.legend()

    ax.set(xlabel='h1', ylabel='max_error',
        title='max_error(h2)')
    ax.grid()

    plt.show()
    
    log_err = np.log(errs)
    log_step = np.log(hxs)
    approximation_order = (log_err[-1] - log_err[0]) / (log_step[-1] - log_step[0])
    print(f'{approximation_order=}')

interact_manual(solve, scheme=[('Метод Либмана', 0), ('Метод Зейделя', 1), ('Метод верхних релаксаций', 2)],
                 a='-2', b='-3', x_begin=x_begin_widget, x_end=x_end_widget, y_begin=y_begin_widget, y_end=y_end_widget, h1='0.05', h2='0.05', w='1.7', current_x=current_x_widget, current_y=current_y_widget)
print()

interactive(children=(Dropdown(description='scheme', options=(('Метод Либмана', 0), ('Метод Зейделя', 1), ('Ме…